# Predict context in Facebook data
Let's predict contextualization in Facebook data using regression on the following variables:

- importance (based on total frequency)
- commitment (based on author posts)
- information (based on post length)
- audience (based on group size)

In [1]:
import pandas as pd
import numpy as np

Load data.

In [2]:
from ast import literal_eval
group_data = pd.read_csv('../../data/facebook-maria/combined_group_data_es_tagged_valid_anchor_group_contain.tsv', sep='\t', index_col=False, converters={'subtree' : literal_eval, 'tree' : literal_eval})
print('%d/%d NE mentions with context'%(group_data.loc[:, 'anchor'].sum(), group_data.shape[0]))
print('%d/%d local NE mentions'%(group_data.loc[:, 'group_contains_NE'].sum(), group_data.shape[0]))

765/18432 NE mentions with context
10131/18432 local NE mentions


In [3]:
def get_text_with_no_context(data, id_var='status_id', txt_var='status_message', subtree_var='subtree', tree_var='tree', context_var='anchor'):
    """
    Replace context from text and return clean text.
    """
    no_context_txt_var = '%s_no_context'%(txt_var)
    data_no_context_txt = []
    for id_i, data_i in data.groupby(id_var):
        txt_i = data_i.loc[:, txt_var].iloc[0]
        # replace context in txt_i
        txt_i_clean = txt_i
        for idx_j, NE_data_j in data_i.iterrows():
            if(NE_data_j.loc[context_var]==1):
    #             print('clean txt before: %s'%(txt_i_clean))
                tree_txt_j = ' '.join([token[0] for token in NE_data_j.loc[tree_var]])
                subtree_txt_j = ' '.join([token[0] for token in NE_data_j.loc[subtree_var]])
                txt_i_clean = txt_i_clean.replace(tree_txt_j, '')
                txt_i_clean = txt_i_clean.replace(subtree_txt_j, '')
        data_i = data_i.assign(**{
            no_context_txt_var : data_i.apply(lambda x: txt_i_clean if x.loc[context_var]==1 else txt_i, axis=1)
        })
        data_no_context_txt.append(data_i)
    data_no_context_txt = pd.concat(data_no_context_txt, axis=0)
    return data_no_context_txt

In [4]:
# importance: NE frequency
NE_var = 'NE_fixed'
NE_counts = group_data.groupby(NE_var).apply(lambda x: x.shape[0]).reset_index().rename(columns={0 : 'NE_count'})
group_data = pd.merge(group_data, NE_counts, on=NE_var)
# information: post length
group_data = get_text_with_no_context(group_data)
txt_var = 'status_message'
no_context_txt_var = '%s_no_context'%(txt_var)
group_data = group_data.assign(**{
    'txt_len_norm' : group_data.loc[:, no_context_txt_var].apply(lambda x: np.log(len(x)+1))
})
# commitment: number of posts per author per group
author_var = 'status_author_id'
group_var = 'group_name'
author_group_counts = group_data.groupby([author_var, group_var]).apply(lambda x: x.shape[0]).reset_index().rename(columns={0 : 'author_group_count'})
group_data = pd.merge(group_data, author_group_counts, on=[author_var, group_var])
# audience: group size
group_counts = group_data.groupby(group_var).apply(lambda x: x.loc[:, author_var].nunique()).reset_index().rename(columns={0 : 'group_size'})
group_data = pd.merge(group_data, group_counts, on=group_var)

Logistic regression on context.

### Predict context all explanatory variables

In [5]:
from sklearn.preprocessing import StandardScaler
## Z-norm scalar vars
scaler = StandardScaler()
group_data_reg = group_data.copy()
# add intercept
group_data_reg = group_data_reg.assign(**{
    'intercept' : 1.
})
scalar_vars = ['NE_count', 'txt_len_norm', 'author_group_count', 'group_size']
for v in scalar_vars:
    group_data_reg = group_data_reg.assign(**{
        v : scaler.fit_transform(group_data_reg.loc[:, v].values.reshape(-1,1))
    })
group_data_reg = group_data_reg.assign(**{
    'group_contains_NE' : group_data_reg.loc[:, 'group_contains_NE'].astype(int)
})

In [6]:
from statsmodels.discrete.discrete_model import Logit
dep_var = 'anchor'
indep_vars = ['NE_count', 'txt_len_norm', 'author_group_count', 'group_size', 'group_contains_NE', 'intercept']
logit_model = Logit(endog=group_data_reg.loc[:, dep_var], exog=group_data_reg.loc[:, indep_vars])
model_results = logit_model.fit()
print(model_results.summary())

Optimization terminated successfully.
         Current function value: 0.166446
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                 anchor   No. Observations:                18432
Model:                          Logit   Df Residuals:                    18426
Method:                           MLE   Df Model:                            5
Date:                Mon, 16 Sep 2019   Pseudo R-squ.:                 0.03618
Time:                        17:55:08   Log-Likelihood:                -3067.9
converged:                       True   LL-Null:                       -3183.1
Covariance Type:            nonrobust   LLR p-value:                 9.066e-48
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
NE_count              -0.2520      0.047     -5.337      0.000      -0.345      -0.159
txt_l

In [7]:
print('dev=%d, df=%d, p=%.3E'%(model_results.llr, model_results.model.df_model, model_results.llr_pvalue))

dev=230, df=5, p=9.066E-48


OK! 

- more NE mentions => less contextualization
- longer text => more context?? because information seeking
- more posts in-group => less context
- bigger group => no effect

## Predict context fixed effect

Let's see if these findings still hold with fixed effects.

We will need to restrict the data to frequent authors, locations, groups for this analysis.

In [8]:
min_author_count = 20
min_NE_count = 20
min_group_count = 20
author_var = 'status_author_id'
NE_var = 'NE_fixed'
group_var = 'group_name'
min_counts = [min_author_count, min_NE_count, min_group_count]
cat_vars = ['status_author_id', 'NE_fixed', 'group_name']
for cat_var, min_count in zip(cat_vars, min_counts):
    cat_counts = group_data_reg.loc[:, cat_var].value_counts()
    group_data_reg = group_data_reg.assign(**{
        '%s_cap'%(cat_var) : group_data_reg.loc[:, cat_var].apply(lambda x: 'RARE' if cat_counts.loc[x] < min_count else x)
    })
    display(group_data_reg.loc[:, '%s_cap'%(cat_var)].value_counts().head(10))

RARE                 10675
10155297402077917      713
10155697137579029      658
10155718325260502      576
10213954436236337      441
10213949852521688      402
10155081537013106      207
10155617866275516      183
10159579876365442      173
1722102481190791       172
Name: status_author_id_cap, dtype: int64

RARE            2930
guayama         1780
coamo           1197
lajas            818
yabucoa          816
barranquitas     814
quebradillas     798
cidra            756
naranjito        695
cayey            472
Name: NE_fixed_cap, dtype: int64

Guayama: Huracán María                              2973
Barranquitas Huracan Maria                          1529
Huracan Maria Coamo                                 1514
Huracan Maria En Lajas                              1016
Quebradillas#Huracan#Maria                           970
Huracan Maria  Yabucoa                               969
HURACAN MARIA CIDRA                                  938
Huracán Maria Vega Alta (Unidos por Vega Alta)       837
Huracan Maria Naranjito                              761
Cayey se levanta ante el paso del huracán María.     704
Name: group_name_cap, dtype: int64

### Find optimal regularization weight

Let's do L2 regularization to discourage variable overfitting.

We'll determine the best weight with log-likelihood comparisons on k-fold cross-validation.

In [ ]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.genmod.families.links import logit
from math import floor, ceil
## likelihood definitions
## logit cdf
def logit_cdf(X):
    return 1 / (1 + np.exp(-X))

## log likelihood
def compute_log_likelihood(params, Y, X):
    q = 2 * Y - 1
    ll = np.sum(np.log(logit_cdf(q * np.dot(X, params))))
    return ll
np.random.seed(123)

# variable definitions
dep_var = 'anchor'
cat_vars = ['NE_fixed', 'status_author_id', 'group_name']
cap_cat_vars = ['%s_cap'%(cat_var) for cat_var in cat_vars]
scalar_vars = ['NE_count', 'txt_len_norm', 'author_group_count', 'group_size', 'group_contains_NE']
indep_formula = ' + '.join(['C(%s)'%(cap_cat_var) for cap_cat_var in cap_cat_vars] + scalar_vars)
formula = '%s ~ %s'%(dep_var, indep_formula)
# convert raw data to exogenous data
# need to do this to force train/test
# to have same features
l2_weights = [0., 0.001, 0.01, 0.1, 1.]
group_data_rand = group_data_reg.copy()
np.random.shuffle(group_data_rand.values)
model_dummy = GLM.from_formula(formula, group_data_rand, family=Binomial(link=logit()))
exog = model_dummy.exog
exog_names = model_dummy.exog_names
endog = model_dummy.endog
# generate cross validation folds
cross_val_folds = 10
N = group_data_rand.shape[0]
cross_val_chunk_size = float(N) / cross_val_folds
cross_val_fold_train_idx = [list(range(int(floor(i*cross_val_chunk_size)), int(ceil((i+1)*cross_val_chunk_size)))) for i in range(cross_val_folds)]
cross_val_fold_test_idx = [list(range(0, int(ceil(i*cross_val_chunk_size)))) + list(range(int(floor((i+1)*cross_val_chunk_size)), N)) for i in range(cross_val_folds)]
weight_likelihoods = []
for l2_weight in l2_weights:
    print('testing weight = %.3f'%(l2_weight))
    likelihoods_l2 = []
    for i, (train_idx_i, test_idx_i) in enumerate(zip(cross_val_fold_train_idx, cross_val_fold_test_idx)):
        print('fold %d'%(i))
        train_XY = group_data_rand.iloc[train_idx_i, :]
        test_X = exog[test_idx_i, :]
        test_Y = endog[test_idx_i]
#         train_i = anchor_data_NE_peak_filter_rand.iloc[train_idx_i, :]
#         test_i = anchor_data_NE_peak_filter_rand.iloc[test_idx_i, :]
        # fit model
        model_i = GLM.from_formula(formula, train_XY, family=Binomial(link=logit()))
        model_res_i = model_i.fit_regularized(maxiter=max_iter, method='elastic_net', alpha=l2_weight, L1_wt=0.)
        # add 0 params for missing coefficients
        # to match X shape
        model_res_i.params = model_res_i.params.loc[exog_names].fillna(0, inplace=False)
        # score test data
        # implement this: http://www.statsmodels.org/stable/_modules/statsmodels/discrete/discrete_model.html#Logit.loglikeobs
        likelihood_i = compute_log_likelihood(model_res_i.params, test_Y, test_X)
        likelihoods_l2.append(likelihood_i)
    weight_likelihoods.append(likelihoods_l2)
weight_likelihoods = pd.DataFrame(np.array(weight_likelihoods), index=l2_weights)

In [ ]:
display(weight_likelihoods.mean(axis=1))

Surprise! The same L2 weight that we found before ($\alpha=0.01$) has the highest log-likelihood.

In [9]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.genmod.families.links import logit
import sys
if('..' not in sys.path):
    sys.path.append('..')
from importlib import reload
import models.model_helpers
reload(models.model_helpers)
from models.model_helpers import compute_err_data, compute_chi2_null_test
dep_var = 'anchor'
cat_vars = ['NE_fixed', 'status_author_id', 'group_name']
# cat_vars = ['NE_fixed', 'status_author_id']
cap_cat_vars = ['%s_cap'%(cat_var) for cat_var in cat_vars]
scalar_vars = ['NE_count', 'txt_len_norm', 'author_group_count', 'group_size', 'group_contains_NE']
indep_formula = ' + '.join(['C(%s)'%(cap_cat_var) for cap_cat_var in cap_cat_vars] + scalar_vars)
formula = '%s ~ %s'%(dep_var, indep_formula)

## non-regularized
# logit_model = Logit.from_formula(formula, data=group_data_reg)
# model_results = logit_model.fit()
## L2 norm to reduce variable inflation on fixed effects
l2_weight = 0.01
max_iter = 100
model_full = GLM.from_formula(formula, group_data_reg, family=Binomial(link=logit()))
model_res_full = model_full.fit_regularized(maxiter=max_iter, method='elastic_net', alpha=l2_weight, L1_wt=0.0)
model_err = compute_err_data(model_res_full)
dev, model_df, p_val = compute_chi2_null_test(model_res_full, group_data_reg, dep_var, max_iter, l2_weight)
print('dev=%.3f, df=%d, p=%.3E'%(dev, model_df, p_val))

/nethome/istewart6/.local/lib/python3.6/site-packages/statsmodels/genmod/generalized_linear_model.py:1303: UserWarning: GLM ridge optimization may have failed, |grad|=0.000192
  warnings.warn(msg)


dev=539.454, df=236, p=8.382E-26


In [10]:
display(model_err.head())
display(model_err.tail())

,mean,SE,z_score,p_val,conf_2.5,conf_97.5
Intercept,-2.029957,28.550367,-0.071101,0.995966,-57.987648,53.927734
C(NE_fixed_cap)[T.aguadilla],0.038792,1.046750,0.037059,0.998904,-2.012800,2.090383
C(NE_fixed_cap)[T.aguas buenas],-0.026400,2.366402,-0.011156,0.999901,-4.664463,4.611663
C(NE_fixed_cap)[T.aibonito],-0.010801,0.854830,-0.012636,0.999873,-1.686238,1.664636
C(NE_fixed_cap)[T.alcaldia],-0.020647,0.800099,-0.025806,0.999469,-1.588812,1.547518


,mean,SE,z_score,p_val,conf_2.5,conf_97.5
NE_count,-0.074550,7.164435,-0.010406,0.999914,-14.116584,13.967484
txt_len_norm,0.036790,0.036339,1.012397,0.305388,-0.034434,0.108013
author_group_count,-0.328241,0.522473,-0.628245,0.693070,-1.352269,0.695787
group_size,0.121074,33.642900,0.003599,0.999990,-65.817799,66.059947
group_contains_NE,-0.623419,0.106352,-5.861851,0.000000,-0.831865,-0.414973


In [17]:
# old
display(model_err.head())
display(model_err.tail())

,mean,SE,z_score,p_val,conf_2.5,conf_97.5
Intercept,-2.029957,28.550367,-0.071101,0.995966,-57.987648,53.927734
C(NE_fixed_cap)[T.aguadilla],0.038792,1.046750,0.037059,0.998904,-2.012800,2.090383
C(NE_fixed_cap)[T.aguas buenas],-0.026400,2.366402,-0.011156,0.999901,-4.664463,4.611663
C(NE_fixed_cap)[T.aibonito],-0.010801,0.854830,-0.012636,0.999873,-1.686238,1.664636
C(NE_fixed_cap)[T.alcaldia],-0.020647,0.800099,-0.025806,0.999469,-1.588812,1.547518


,mean,SE,z_score,p_val,conf_2.5,conf_97.5
NE_count,-0.074550,7.164435,-0.010406,0.999914,-14.116584,13.967484
txt_len_norm,0.036790,0.036339,1.012397,0.305388,-0.034434,0.108013
author_group_count,-0.328241,0.522473,-0.628245,0.693070,-1.352269,0.695787
group_size,0.121074,33.642900,0.003599,0.999990,-65.817799,66.059947
group_contains_NE,-0.623419,0.106352,-5.861851,0.000000,-0.831865,-0.414973


Not sure if I believe these results, given that the intercept is insignificant when it should be negative. One reason for a negative intercept is that it serves as the "base" category for all the fixed effects.

If these results hold, then `txt_len_norm` is the only significant result which hurts our use of the other explanatory variables in later analysis.

**Conclusion**: this model is severely overfitting due to dependent variable sparsity.

What is the deviance etc. for a model with only the fixed effects?

In [18]:
dep_var = 'anchor'
cat_vars = ['NE_fixed', 'status_author_id', 'group_name']
cap_cat_vars = ['%s_cap'%(cat_var) for cat_var in cat_vars]
indep_formula = ' + '.join(['C(%s)'%(cap_cat_var) for cap_cat_var in cap_cat_vars])
formula = '%s ~ %s'%(dep_var, indep_formula)

## non-regularized
# logit_model = Logit.from_formula(formula, data=group_data_reg)
# model_results = logit_model.fit()
## L2 norm to reduce variable inflation on fixed effects
l2_weight = 0.01
max_iter = 100
model_full = GLM.from_formula(formula, group_data_reg, family=Binomial(link=logit()))
model_res_full = model_full.fit_regularized(maxiter=max_iter, method='elastic_net', alpha=l2_weight, L1_wt=0.0)
model_err = compute_err_data(model_res_full)
dev, model_df, p_val = compute_chi2_null_test(model_res_full, group_data_reg, dep_var, max_iter, l2_weight)
print('dev=%.3f, df=%d, p=%.3E'%(dev, model_df, p_val))

dev=397.227, df=231, p=6.258E-11


Sanity check: the full model has higher deviance than the model with only fixed effects.

What if we drop the fixed effects and just fit the explanatory variables?

In [19]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.genmod.families.links import logit
import sys
if('..' not in sys.path):
    sys.path.append('..')
from importlib import reload
import models.model_helpers
reload(models.model_helpers)
from models.model_helpers import compute_err_data, compute_chi2_null_test
dep_var = 'anchor'
scalar_vars = ['NE_count', 'txt_len_norm', 'author_group_count', 'group_size', 'group_contains_NE']
indep_formula = ' + '.join(scalar_vars)
formula = '%s ~ %s'%(dep_var, indep_formula)

## L2 norm to reduce variable inflation on fixed effects
l2_weight = 0.01
max_iter = 100
model_full = GLM.from_formula(formula, group_data_reg, family=Binomial(link=logit()))
model_res_full = model_full.fit_regularized(maxiter=max_iter, method='elastic_net', alpha=l2_weight, L1_wt=0.0)
model_err = compute_err_data(model_res_full)
dev, model_df, p_val = compute_chi2_null_test(model_res_full, group_data_reg, dep_var, max_iter, l2_weight)
print('dev=%.3f, df=%d, p=%.3E'%(dev, model_df, p_val))

dev=183.360, df=5, p=1.025E-37


In [20]:
display(model_err)

,mean,SE,z_score,p_val,conf_2.5,conf_97.5
Intercept,-2.372707,0.040322,-58.843649,0.000000,-2.451737,-2.293677
NE_count,-0.074842,0.035250,-2.123216,0.000007,-0.143930,-0.005755
txt_len_norm,0.093893,0.030279,3.100962,0.000000,0.034548,0.153237
author_group_count,-0.224318,0.040297,-5.566633,0.000000,-0.303299,-0.145338
group_size,0.046145,0.031911,1.446066,0.036518,-0.016399,0.108689
group_contains_NE,-0.680379,0.064262,-10.587623,0.000000,-0.806330,-0.554428


### Predict context fixed effect from containment

Simpler: let's do the same thing but just using `group_contains_NE`.

In [118]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.genmod.families.links import logit
import sys
if('..' not in sys.path):
    sys.path.append('..')
from models.model_helpers import compute_err_data
dep_var = 'anchor'
cat_vars = ['status_author_id', 'group_name']
cap_cat_vars = ['%s_cap'%(cat_var) for cat_var in cat_vars]
scalar_vars = ['group_contains_NE']
indep_formula = ' + '.join(['C(%s)'%(cap_cat_var) for cap_cat_var in cap_cat_vars] + scalar_vars)
formula = '%s ~ %s'%(dep_var, indep_formula)

## non-regularized
# logit_model = Logit.from_formula(formula, data=group_data_reg)
# model_results = logit_model.fit()
## L2 norm to reduce variable inflation on fixed effects
l2_weight = 0.01
max_iter = 100
model_full = GLM.from_formula(formula, group_data_reg, family=Binomial(link=logit()))
model_res_full = model_full.fit_regularized(maxiter=max_iter, method='elastic_net', alpha=l2_weight, L1_wt=0.0)
model_err = compute_err_data(model_res_full)
dev, model_df, p_val = compute_chi2_null_test(model_res_full, group_data_reg, dep_var, max_iter, l2_weight)
print('dev=%.3f, df=%d, p=%.3E'%(dev, model_df, p_val))

/nethome/istewart6/.local/lib/python3.6/site-packages/statsmodels/genmod/generalized_linear_model.py:1303: UserWarning: GLM ridge optimization may have failed, |grad|=0.000036
  warnings.warn(msg)


formula=anchor ~ 1
dev=248.732, df=127, p=6.416E-10


In [119]:
display(model_err.head())
display(model_err.tail())

,mean,SE,z_score,p_val,conf_2.5,conf_97.5
Intercept,-2.115682,0.544714,-3.884024,0.000000,-3.183301,-1.048062
C(status_author_id_cap)[T.370781346691489],-0.007173,0.968358,-0.007407,0.999956,-1.905120,1.890774
C(status_author_id_cap)[T.377022969399263],-0.011229,0.842238,-0.013332,0.999858,-1.661984,1.639527
C(status_author_id_cap)[T.530249124001312],-0.008098,0.945828,-0.008561,0.999942,-1.861887,1.845692
C(status_author_id_cap)[T.603616776695697],-0.063274,0.567813,-0.111434,0.990093,-1.176167,1.049620


,mean,SE,z_score,p_val,conf_2.5,conf_97.5
C(group_name_cap)[T.Quebradillas#Huracan#Maria],-0.179122,0.351366,-0.509786,0.794955,-0.867786,0.509543
C(group_name_cap)[T.RARE],0.042371,0.442223,0.095814,0.992675,-0.824370,0.909112
C(group_name_cap)[T.Se Buscan Mayaguez PR Huracan Maria],0.009164,0.535183,0.017123,0.999766,-1.039775,1.058103
C(group_name_cap)[T.Updates de Familia en Isabela y Huracan Maria],-0.035516,0.514561,-0.069023,0.996199,-1.044038,0.973005
group_contains_NE,-0.657564,0.077184,-8.519425,0.000000,-0.808842,-0.506286


This makes more sense. The local effect is still strong.